In [7]:
from __future__ import absolute_import, division, print_function, unicode_literals
# import libraries
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.models import Sequential 
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers

# Helper libraries
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from google.colab import files
import io

#for greg
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score


ytrain_byteseq = files.upload() # choose 'ytrain.npy' from local system
ytrain_filelike = io.BytesIO(ytrain_byteseq['ytrain.npy']) # create file-like object
ytrain = np.load(ytrain_filelike,allow_pickle=True) # create regular numpy array
xtrain_byteseq = files.upload() # choose 'xtrain.npy' from local system
xtrain_filelike = io.BytesIO(xtrain_byteseq['xtrain.npy']) # create file-like object
xtrain = np.load(xtrain_filelike,allow_pickle=True) # create regular numpy array

yval_byteseq = files.upload() # choose 'yval.npy' from local system
yval_filelike = io.BytesIO(yval_byteseq['yval.npy']) # create file-like object
yval = np.load(yval_filelike,allow_pickle=True) # create regular numpy array
xval_byteseq = files.upload() # choose 'xval.npy' from local system
xval_filelike = io.BytesIO(xval_byteseq['xval.npy']) # create file-like object
xval = np.load(xval_filelike,allow_pickle=True) # create regular numpy array

ytest_byteseq = files.upload() # choose 'ytest.npy' from local system
ytest_filelike = io.BytesIO(ytest_byteseq['ytest.npy']) # create file-like object
ytest = np.load(ytest_filelike,allow_pickle=True) # create regular numpy array
xtest_byteseq = files.upload() # choose 'ytest.npy' from local system
xtest_filelike = io.BytesIO(xtest_byteseq['xtest.npy']) # create file-like object
xtest = np.load(xtest_filelike,allow_pickle=True) # create regular numpy array

Saving ytrain.npy to ytrain (3).npy


Saving xtrain.npy to xtrain (2).npy


Saving yval.npy to yval (2).npy


Saving xval.npy to xval (2).npy


Saving ytest.npy to ytest (3).npy


Saving xtest.npy to xtest (2).npy


In [14]:
# the keys for the meaning of each integer label in test sets
'''
feature_map = {'Rock':0,'Electronic':1,'Experimental':2,'Hip-Hop':3,'Folk':4,'Instrumental':5,'Pop':6,
              'International':7,'Classical':8,'Old-Time / Historic':9, 'Jazz':10,'Country':11,'Soul-RnB':12,
              'Spoken':13,'Blues':14,'Easy Listening':15}
'''
ytrain_1hot = to_categorical(ytrain).astype('int')
yval_1hot = to_categorical(yval).astype('int')
ytest_1hot = to_categorical(ytest).astype('int')

# make full trainval set for alternate splits later
xtrainval = np.concatenate((xtrain,xval),axis=0)
ytrainval = np.concatenate((ytrain,yval),axis=0)
ytrainval_1hot = to_categorical(ytrainval).astype('int') 

# break sets up by features for investigation in smaller groups
chroma_cens = xtrain[:,:84] # length 12
chroma_cqt = xtrain[:,84:168] # length 12
chroma_stft = xtrain[:,168:252]# length 12
chroma_mfcc = xtrain[:,252:392] # length 20
rmse = xtrain[:,392:399]
spectral_bandwidth = xtrain[:,399:406]
spectral_centroid = xtrain[:,406:413]
spectral_contrast = xtrain[:,413:462]# length 7
spectral_rolloff = xtrain[:,462:469] 
tonnetz = xtrain[:,469:511] # length 6
zcr = xtrain[:,511:518]
dates = xtrain[:,518]
duration = xtrain[:,519]

chroma_censtrainval = xtrainval[:,:84] # length 12
chroma_cqttrainval = xtrainval[:,84:168] # length 12
chroma_stfttrainval = xtrainval[:,168:252]# length 12
chroma_mfcctrainval = xtrainval[:,252:392] # length 20
rmsetrainval = xtrainval[:,392:399]
spectral_bandwidthtrainval = xtrainval[:,399:406]
spectral_centroidtrainval = xtrainval[:,406:413]
spectral_contrasttrainval = xtrainval[:,413:462]# length 7
spectral_rollofftrainval = xtrainval[:,462:469] 
tonnetztrainval = xtrainval[:,469:511] # length 6
zcrtrainval = xtrainval[:,511:518]
datestrainval = xtrainval[:,518]
durationtrainval = xtrainval[:,519]

chroma_censval = xval[:,:84] # length 12
chroma_cqtval = xval[:,84:168] # length 12
chroma_stftval = xval[:,168:252]# length 12
chroma_mfccval = xval[:,252:392] # length 20
rmseval = xval[:,392:399]
spectral_bandwidthval = xval[:,399:406]
spectral_centroidval = xval[:,406:413]
spectral_contrastval = xval[:,413:462]# length 7
spectral_rolloffval = xval[:,462:469] 
tonnetzval = xval[:,469:511] # length 6
zcrval = xval[:,511:518]
datesval = xval[:,518]
durationval = xval[:,519]

# reorder features for compatibility with CNN

# currently, the features are a timeseries for each statistic
# but we want to reindex so that all statistics are intermingled
# but grouped by time
# this function will help with that
def get_index_order(subinterval_length):
    idxs = []
    for i in range(1,subinterval_length+1):
        idx = []
        for j in range(1,8):
            idx.append(subinterval_length*(j-1)+i-1)
        idxs.append(idx)
    idxs = np.array(idxs).flatten()
    return idxs

order_12 = get_index_order(12)
order_20 = get_index_order(20)
order_7 = get_index_order(7)
order_6 = get_index_order(6)

# reindex timeseries arrays
chroma_cens = chroma_cens[:,order_12] # length 12
chroma_cqt = chroma_cqt[:,order_12] # length 12
chroma_stft = chroma_stft[:,order_12] # length 20
chroma_mfcc = chroma_mfcc[:,order_20] # length 20
spectral_contrast = spectral_contrast[:,order_7] # length 7
tonnetz = tonnetz[:,order_6] # length 6

chroma_censval = chroma_censval[:,order_12] # length 12
chroma_cqtval = chroma_cqtval[:,order_12] # length 12
chroma_stftval = chroma_stftval[:,order_12] # length 20
chroma_mfccval = chroma_mfccval[:,order_20] # length 20
spectral_contrastval = spectral_contrastval[:,order_7] # length 7
tonnetzval = tonnetzval[:,order_6] # length 6

chroma_censtrainval = chroma_censtrainval[:,order_12] # length 12
chroma_cqttrainval = chroma_cqttrainval[:,order_12] # length 12
chroma_stfttrainval = chroma_stfttrainval[:,order_12] # length 20
chroma_mfcctrainval = chroma_mfcctrainval[:,order_20] # length 20
spectral_contrasttrainval = spectral_contrasttrainval[:,order_7] # length 7
tonnetztrainval = tonnetztrainval[:,order_6] # length 6

#reshape for cnn
cnn_chroma_cens = chroma_cens.reshape(xtrain.shape[0],int(chroma_cens.shape[1]/7),7)
cnn_chroma_cqt = chroma_cqt.reshape(xtrain.shape[0],int(chroma_cqt.shape[1]/7),7)
cnn_chroma_stft = chroma_stft.reshape(xtrain.shape[0],int(chroma_stft.shape[1]/7),7)
cnn_chroma_mfcc = chroma_mfcc.reshape(xtrain.shape[0],int(chroma_mfcc.shape[1]/7),7)
cnn_rmse = rmse.reshape(xtrain.shape[0],int(rmse.shape[1]/7),7)
cnn_spectral_bandwidth = spectral_bandwidth.reshape(xtrain.shape[0],int(spectral_bandwidth.shape[1]/7),7)
cnn_spectral_centroid = spectral_centroid.reshape(xtrain.shape[0],int(spectral_centroid.shape[1]/7),7)
cnn_spectral_contrast = spectral_contrast.reshape(xtrain.shape[0],int(spectral_contrast.shape[1]/7),7)
cnn_spectral_rolloff = spectral_rolloff.reshape(xtrain.shape[0],int(spectral_rolloff.shape[1]/7),7)
cnn_tonnetz = tonnetz.reshape(xtrain.shape[0],int(tonnetz.shape[1]/7),7)
cnn_zcr = zcr.reshape(xtrain.shape[0],int(zcr.shape[1]/7),7)

cnn_chroma_censval = chroma_censval.reshape(xval.shape[0],int(chroma_censval.shape[1]/7),7)
cnn_chroma_cqtval = chroma_cqtval.reshape(xval.shape[0],int(chroma_cqtval.shape[1]/7),7)
cnn_chroma_stftval = chroma_stftval.reshape(xval.shape[0],int(chroma_stftval.shape[1]/7),7)
cnn_chroma_mfccval = chroma_mfccval.reshape(xval.shape[0],int(chroma_mfccval.shape[1]/7),7)
cnn_rmseval = rmseval.reshape(xval.shape[0],int(rmseval.shape[1]/7),7)
cnn_spectral_bandwidthval = spectral_bandwidthval.reshape(xval.shape[0],int(spectral_bandwidthval.shape[1]/7),7)
cnn_spectral_centroidval = spectral_centroidval.reshape(xval.shape[0],int(spectral_centroidval.shape[1]/7),7)
cnn_spectral_contrastval = spectral_contrastval.reshape(xval.shape[0],int(spectral_contrastval.shape[1]/7),7)
cnn_spectral_rolloffval = spectral_rolloffval.reshape(xval.shape[0],int(spectral_rolloffval.shape[1]/7),7)
cnn_tonnetzval = tonnetzval.reshape(xval.shape[0],int(tonnetzval.shape[1]/7),7)
cnn_zcrval = zcrval.reshape(xval.shape[0],int(zcrval.shape[1]/7),7)

cnn_chroma_censtrainval = chroma_censtrainval.reshape(xtrainval.shape[0],int(chroma_censtrainval.shape[1]/7),7)
cnn_chroma_cqttrainval = chroma_cqttrainval.reshape(xtrainval.shape[0],int(chroma_cqttrainval.shape[1]/7),7)
cnn_chroma_stfttrainval = chroma_stfttrainval.reshape(xtrainval.shape[0],int(chroma_stfttrainval.shape[1]/7),7)
cnn_chroma_mfcctrainval = chroma_mfcctrainval.reshape(xtrainval.shape[0],int(chroma_mfcctrainval.shape[1]/7),7)
cnn_rmsetrainval = rmsetrainval.reshape(xtrainval.shape[0],int(rmsetrainval.shape[1]/7),7)
cnn_spectral_bandwidthtrainval = spectral_bandwidthtrainval.reshape(xtrainval.shape[0],int(spectral_bandwidthtrainval.shape[1]/7),7)
cnn_spectral_centroidtrainval = spectral_centroidtrainval.reshape(xtrainval.shape[0],int(spectral_centroidtrainval.shape[1]/7),7)
cnn_spectral_contrasttrainval = spectral_contrasttrainval.reshape(xtrainval.shape[0],int(spectral_contrasttrainval.shape[1]/7),7)
cnn_spectral_rollofftrainval = spectral_rollofftrainval.reshape(xtrainval.shape[0],int(spectral_rollofftrainval.shape[1]/7),7)
cnn_tonnetztrainval = tonnetztrainval.reshape(xtrainval.shape[0],int(tonnetztrainval.shape[1]/7),7)
cnn_zcrtrainval = zcrtrainval.reshape(xtrainval.shape[0],int(zcrtrainval.shape[1]/7),7)

# print number of songs in each category
print(np.sum(ytrain_1hot,axis=0))
print(np.sum(yval_1hot,axis=0))
print(np.sum(ytest_1hot,axis=0))

[11518  7564  8585  2925  2246  1658  1898  1139   983   436   458   160
   142   354    87    21]
[1240  868  995  273  263  197  217  126  110   53   56   13   16   29
    7    1]
[1424  940 1028  354  294  224  217  124  137   65   57   21   17   40
   16    2]


In [25]:
# train and test best MLP architecture using all features 
layer_list = [ #520 parameters in input
          layers.BatchNormalization(),
          layers.Dropout(rate=0.2),
          layers.Dense(512,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dropout(rate=0.2),
          layers.Dense(256,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(128,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(64,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(32,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,activation='softmax',kernel_initializer='lecun_normal'),
          ]
model = Sequential(layer_list)
model.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001,decay_steps=2000,decay_rate=0.99)),
    loss="categorical_crossentropy", metrics=['accuracy'],)
history = model.fit(xtrain, ytrain_1hot, batch_size=32, epochs=100, validation_data=(xval, yval_1hot))
from sklearn import preprocessing
#scaler = preprocessing.StandardScaler().fit(xtest)
#xtest_scaled = scaler.transform(xtest)
#model.evaluate(xtest_scaled,ytest_1hot)
'''
model.evaluate(xtest,ytest_1hot)
test_predictions = np.rint(model.predict(xtest))
print(test_predictions)
test_predictions = np.sum(test_predictions,axis=0)
truth = np.sum(ytest_1hot, axis=0)
print(test_predictions)
print(truth)
'''

Epoch 1/100
1256/1256 [==============================] - 21s 15ms/step - loss: 1.4853 - accuracy: 0.5385 - val_loss: 1.2914 - val_accuracy: 0.5717
Epoch 2/100
1256/1256 [==============================] - 18s 14ms/step - loss: 1.3034 - accuracy: 0.5731 - val_loss: 1.2270 - val_accuracy: 0.5966
Epoch 3/100
1256/1256 [==============================] - 18s 15ms/step - loss: 1.2728 - accuracy: 0.5844 - val_loss: 1.2298 - val_accuracy: 0.5901
Epoch 4/100
1256/1256 [==============================] - 18s 15ms/step - loss: 1.2523 - accuracy: 0.5877 - val_loss: 1.1854 - val_accuracy: 0.6129
Epoch 5/100
1256/1256 [==============================] - 18s 15ms/step - loss: 1.2340 - accuracy: 0.5967 - val_loss: 1.1947 - val_accuracy: 0.6093
Epoch 6/100
1256/1256 [==============================] - 18s 14ms/step - loss: 1.2146 - accuracy: 0.6020 - val_loss: 1.1696 - val_accuracy: 0.6216
Epoch 7/100
1256/1256 [==============================] - 18s 14ms/step - loss: 1.2025 - accuracy: 0.6026 - val_loss: 1

In [29]:
scaler = preprocessing.StandardScaler().fit(xtest)
xtest_scaled = scaler.transform(xtest)
model.evaluate(xtest_scaled,ytest_1hot)


155/155 [==============================] - 1s 7ms/step - loss: 1.1034 - accuracy: 0.6665


[1.1033774614334106, 0.6665322780609131]

In [30]:
layer_list = [ #520 parameters in input
          layers.BatchNormalization(),
          layers.Dense(50,kernel_initializer='lecun_normal'), #let's try dimensionality reduction
          layers.BatchNormalization(),
          layers.Activation('relu'),
          layers.Dense(512,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('relu'),
          layers.Dense(16,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,activation='softmax',kernel_initializer='lecun_normal'),
          ]
model = Sequential(layer_list)
model.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001,decay_steps=2000,decay_rate=0.99)),
    loss="categorical_crossentropy", metrics=['accuracy'],)
history = model.fit(xtrain, ytrain_1hot, batch_size=32, epochs=50, validation_data=(xval, yval_1hot))
model.evaluate(xtest_scaled,ytest_1hot)

Epoch 1/50
1256/1256 [==============================] - 8s 5ms/step - loss: 1.5409 - accuracy: 0.5393 - val_loss: 1.3485 - val_accuracy: 0.5609
Epoch 2/50
1256/1256 [==============================] - 6s 5ms/step - loss: 1.2532 - accuracy: 0.5984 - val_loss: 1.2405 - val_accuracy: 0.5977
Epoch 3/50
1256/1256 [==============================] - 6s 5ms/step - loss: 1.1933 - accuracy: 0.6154 - val_loss: 1.1705 - val_accuracy: 0.6268
Epoch 4/50
1256/1256 [==============================] - 7s 5ms/step - loss: 1.1660 - accuracy: 0.6217 - val_loss: 1.1664 - val_accuracy: 0.6203
Epoch 5/50
1256/1256 [==============================] - 6s 5ms/step - loss: 1.1386 - accuracy: 0.6333 - val_loss: 1.1345 - val_accuracy: 0.6272
Epoch 6/50
1256/1256 [==============================] - 6s 5ms/step - loss: 1.1173 - accuracy: 0.6390 - val_loss: 1.1448 - val_accuracy: 0.6192
Epoch 7/50
1256/1256 [==============================] - 6s 5ms/step - loss: 1.0992 - accuracy: 0.6444 - val_loss: 1.1294 - val_accuracy:

[1.1949912309646606, 0.635887086391449]

In [38]:
# training the same model on a smaller subset of features produces worse results on the validation set
layer_list2 = [ #520 parameters in input
          layers.BatchNormalization(),
          layers.Dense(50,kernel_initializer='lecun_normal'), #let's try dimensionality reduction
          layers.BatchNormalization(),
          layers.Activation('relu'),
          layers.Dense(512,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('relu'),
          layers.Dense(16,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,activation='softmax',kernel_initializer='lecun_normal'),
          ]
chroma_censtest = cnn_chroma_censtest.reshape(cnn_chroma_censtest.shape[0],84)
print(chroma_cens.shape)
print(chroma_censval.shape)
print(cnn_chroma_censtest.shape)
model2 = Sequential(layer_list2)
model2.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001,decay_steps=2000,decay_rate=0.99)),
    loss="categorical_crossentropy", metrics=['accuracy'],)
history = model2.fit(chroma_cens, ytrain_1hot, batch_size=32, epochs=20, validation_data=(chroma_censval, yval_1hot))
scaler = preprocessing.StandardScaler().fit(chroma_censtest) #defined test set below, not actually doing cnn here
chroma_censtest_scaled = scaler.transform(chroma_censtest)
model2.evaluate(chroma_censtest_scaled,ytest_1hot)

(40174, 84)
(4464, 84)
(4960, 12, 7)
Epoch 1/20
1256/1256 [==============================] - 8s 5ms/step - loss: 2.0058 - accuracy: 0.3329 - val_loss: 1.8572 - val_accuracy: 0.3401
Epoch 2/20
1256/1256 [==============================] - 6s 5ms/step - loss: 1.7940 - accuracy: 0.3750 - val_loss: 1.8121 - val_accuracy: 0.3651
Epoch 3/20
1256/1256 [==============================] - 6s 5ms/step - loss: 1.7573 - accuracy: 0.3866 - val_loss: 1.7802 - val_accuracy: 0.3880
Epoch 4/20
1256/1256 [==============================] - 6s 4ms/step - loss: 1.7375 - accuracy: 0.3913 - val_loss: 1.8071 - val_accuracy: 0.3833
Epoch 5/20
1256/1256 [==============================] - 6s 5ms/step - loss: 1.7263 - accuracy: 0.3991 - val_loss: 1.7804 - val_accuracy: 0.3920
Epoch 6/20
1256/1256 [==============================] - 6s 5ms/step - loss: 1.7116 - accuracy: 0.4069 - val_loss: 1.7596 - val_accuracy: 0.4032
Epoch 7/20
1256/1256 [==============================] - 6s 4ms/step - loss: 1.7018 - accuracy: 0.40

[1.7049670219421387, 0.4145161211490631]

In [31]:
# realizing that I forgot to divide test set up by features when preprocessing
chroma_censtest = xtest[:,:84] # length 12
chroma_cqttest = xtest[:,84:168] # length 12
chroma_stfttest = xtest[:,168:252]# length 12
chroma_mfcctest = xtest[:,252:392] # length 20
rmsetest = xtest[:,392:399]
spectral_bandwidthtest = xtest[:,399:406]
spectral_centroidtest = xtest[:,406:413]
spectral_contrasttest = xtest[:,413:462]# length 7
spectral_rollofftest = xtest[:,462:469] 
tonnetztest = xtest[:,469:511] # length 6
zcrtest = xtest[:,511:518]
datestest = xtest[:,518]
durationtest = xtest[:,519]

# reorder features for compatibility with CNN
order_12 = get_index_order(12)
order_20 = get_index_order(20)
order_7 = get_index_order(7)
order_6 = get_index_order(6)

# reindex timeseries arrays
chroma_censtest = chroma_censtest[:,order_12] # length 12
chroma_cqttest = chroma_cqttest[:,order_12] # length 12
chroma_stfttest = chroma_stfttest[:,order_12] # length 20
chroma_mfcctest = chroma_mfcctest[:,order_20] # length 20
spectral_contrasttest = spectral_contrasttest[:,order_7] # length 7
tonnetztest = tonnetztest[:,order_6] # length 6

# reshape
cnn_chroma_censtest = chroma_censtest.reshape(xtest.shape[0],int(chroma_censtest.shape[1]/7),7)
cnn_chroma_cqttest = chroma_cqttest.reshape(xtest.shape[0],int(chroma_cqttest.shape[1]/7),7)
cnn_chroma_stfttest = chroma_stfttest.reshape(xtest.shape[0],int(chroma_stfttest.shape[1]/7),7)
cnn_chroma_mfcctest = chroma_mfcctest.reshape(xtest.shape[0],int(chroma_mfcctest.shape[1]/7),7)
cnn_rmsetest = rmsetest.reshape(xtest.shape[0],int(rmsetest.shape[1]/7),7)
cnn_spectral_bandwidthtest = spectral_bandwidthtest.reshape(xtest.shape[0],int(spectral_bandwidthtest.shape[1]/7),7)
cnn_spectral_centroidtest = spectral_centroidtest.reshape(xtest.shape[0],int(spectral_centroidtest.shape[1]/7),7)
cnn_spectral_contrasttest = spectral_contrasttest.reshape(xtest.shape[0],int(spectral_contrasttest.shape[1]/7),7)
cnn_spectral_rollofftest = spectral_rollofftest.reshape(xtest.shape[0],int(spectral_rollofftest.shape[1]/7),7)
cnn_tonnetztest = tonnetztest.reshape(xtest.shape[0],int(tonnetztest.shape[1]/7),7)
cnn_zcrtest = zcrtest.reshape(xtest.shape[0],int(zcrtest.shape[1]/7),7)

In [50]:
# let's take a look at cnn
print(cnn_chroma_censtest.shape)
layer_list3 = [
          layers.BatchNormalization(),
          layers.Conv1D(7,1,strides=1,kernel_initializer='lecun_normal',input_shape=(chroma_cens.shape[1],7)), # each of 7 statistics is kind of like its own channel!
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Conv1D(8,1,strides=1,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Flatten(), # should have 160 if length is 20?
          layers.BatchNormalization(),
          layers.Dropout(rate=0.2),
          layers.Dense(80,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(40,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(32,kernel_initializer='lecun_normal'),
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,activation='softmax',kernel_initializer='lecun_normal'),
          ]
model3 = Sequential(layer_list3)
model3.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001,decay_steps=2000,decay_rate=0.99)),
    loss="categorical_crossentropy", metrics=['accuracy'],)
history = model3.fit(cnn_chroma_cens, ytrain_1hot, batch_size=32, epochs=50, validation_data=(cnn_chroma_censval, yval_1hot))
scaler = preprocessing.StandardScaler().fit(cnn_chroma_censtest.reshape(cnn_chroma_censtest.shape[0],84)) #standard scaler needs 2d data
cnn_chroma_censtest_scaled = scaler.transform(cnn_chroma_censtest.reshape(cnn_chroma_censtest.shape[0],84))
# had some technical difficulties getting test accuracy
#model2.evaluate(cnn_chroma_censtest_scaled.reshape(cnn_chroma_censtest_scaled.shape[0],12,7),ytest_1hot)

(4960, 12, 7)
Epoch 1/50
1256/1256 [==============================] - 9s 5ms/step - loss: 2.0106 - accuracy: 0.3133 - val_loss: 1.8353 - val_accuracy: 0.3311
Epoch 2/50
1256/1256 [==============================] - 6s 5ms/step - loss: 1.8267 - accuracy: 0.3540 - val_loss: 1.8202 - val_accuracy: 0.3304
Epoch 3/50
1256/1256 [==============================] - 6s 5ms/step - loss: 1.8023 - accuracy: 0.3625 - val_loss: 1.7838 - val_accuracy: 0.3562
Epoch 4/50
1256/1256 [==============================] - 6s 5ms/step - loss: 1.7851 - accuracy: 0.3728 - val_loss: 1.7730 - val_accuracy: 0.3642
Epoch 5/50
1256/1256 [==============================] - 6s 5ms/step - loss: 1.7641 - accuracy: 0.3825 - val_loss: 1.7390 - val_accuracy: 0.3965
Epoch 6/50
1256/1256 [==============================] - 6s 5ms/step - loss: 1.7513 - accuracy: 0.3877 - val_loss: 1.7475 - val_accuracy: 0.3869
Epoch 7/50
1256/1256 [==============================] - 6s 5ms/step - loss: 1.7380 - accuracy: 0.3936 - val_loss: 1.7027 -

In [53]:
'''
from sklearn.utils import shuffle
ytrainval = ytrainval.reshape(ytrainval.shape[0],1)
xtrainval = shuffle(xtrainval)
indices = [] 
num_counter = []
for i in range(16):
    num_counter.append(0)
train_max_list = [1000,1000,1000,1000,1000,1000,1000,1000,1000,100,100,100,100,100,100,20]
for i in range(xtrainval.shape[0]):
    if num_counter[int(xtrainval[i,-1])] < train_max_list[int(xtrainval[i,-1])]:
        num_counter[int(xtrainval[i,-1])] += 1
        indices.append(i)
print(num_counter)
both_train_small = xtrainval[indices]
y_train_small = both_train_small[:,-1]
y_train_small_1hot = to_categorical(y_train_small).astype('int')
x_train_small = both_train_small[:,:-1]
print(y_train_small_1hot.shape)
print(y_train_small.shape)
print(x_train_small.shape)
'''

IndexError: ignored

In [ ]:
'''
layer_list = [ #520 parameters in input
          layers.BatchNormalization(),
          layers.Dropout(0.4),
          layers.Dense(10,kernel_initializer='lecun_normal'), #let's try dimensionality reduction
          layers.BatchNormalization(),
          layers.Activation('selu'),
          layers.Dense(16,activation='softmax',kernel_initializer='lecun_normal'),
          ]
model = Sequential(layer_list)
model.compile(optimizer=keras.optimizers.Adam(
    learning_rate=keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=0.001,decay_steps=2000,decay_rate=0.99)),
    loss="categorical_crossentropy", metrics=['accuracy'],)
history = model.fit(x_train_small, y_train_small_1hot, batch_size=32, epochs=400, validation_data=(x_val_small, y_val_small_1hot))
'''